<span style="color:blue">
    
---
# Analysis of data on the elections in Georgia 2020-11-03
    
---
    
</span>    

In [1]:
import os
import sys
import inspect
import warnings
import datetime
import time
import pdb
import xml
import re
import openpyxl
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotnine
from tqdm import tqdm
import collections
import zipfile
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select

def whoami():
    return sys._getframe(1).f_code.co_name
now = datetime.datetime.now
now()

datetime.datetime(2020, 12, 25, 2, 23, 48, 564118)

<span style="color:blue">
    
---
## Selenium setup
#### for automated download of data files


---    
    
 </span></span>

In [2]:
# set download directory
def chrome_options_download_dir(ddir):
    if not os.path.isdir(ddir):
        os.mkdir(ddir)
    prefs = {"download.default_directory" : os.path.abspath(ddir)}

    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs",prefs)
    options.add_argument("download.default_directory="+os.path.abspath(ddir))
    return options

<span style="color:blue">
    
---

## Results by County and VoteType

https://results.enr.clarityelections.com/GA/105369/web.264614/#/summary   
    or  
https://results.enr.clarityelections.com/GA/105369/web.264614/#/detail/10000    


another possible source  for the senate race
    
www.nbcnews.com/politics/2020-special-elections/georgia-senate-results
        
    
---

</span>

In [3]:

fpath = "./data/detailxml/detail.xml"
import xml.etree.ElementTree as et 
def node_msg(node, pref= '', tag=True, attribs=None):
    if attribs is None:
        attribs = []
    if isinstance(attribs, str):
        attribs = [attribs]
    msg = ""
    msg += "{0}{1}  tag: {2}".format(pref,node, node.tag)
    pref = pref+"  "    
    for at in attribs:
        atval = None
        if at in node.attrib.keys():
            atval = node.attrib[at]
        msg += "\n{0}{1} '{2}'".format(pref, at, atval)            
    return msg

In [4]:
CountyVoteTup = collections.namedtuple("CountyVoteTup",['CountyName', 'Votes'])
VoteTypeTup = collections.namedtuple("VoteTypeTup",    ['CountyName', 'Votes', 'VoteType'])    
ChoiceTup = collections.namedtuple("ChoiceTup",        ['CountyName', 'Votes', 'VoteType','Choice'])   
ContestTup = collections.namedtuple("ContestTup",      ['CountyName', 'Votes', 'VoteType','Choice', 'Contest'])  
def doCountyVote(node):
    county_name = node.attrib['name']
    votes = int(node.attrib['votes'])
    cv_tup = CountyVoteTup(county_name, votes)
    return cv_tup
def doVoteType(root_node, pref='', verbosity=0):
    if verbosity > 0:
        print(node_msg(root_node, pref=pref+'  ', attribs='text'))
    VoteTypeName = root_node.attrib['name']        
    counties = []
    votetype_tups = []
    for node in root_node:      
        if node.tag == "County":
            cv = doCountyVote(node)
            vt = VoteTypeTup(*cv, VoteTypeName)
            votetype_tups.append(vt)
            counties.append(cv)
    res = pd.DataFrame(counties)
    res.set_index("CountyName", inplace=True)
    res.rename(columns={"Votes":VoteTypeName}, inplace=True)
    return (res, votetype_tups)
def doChoice(root_node, pref='', verbosity=0):
    if verbosity > 0:
        print(node_msg(root_node, pref=pref+'  ', attribs='text'))   
    ChoiceText = root_node.attrib['text']         
    res = None
    choice_tups = []
    for node in root_node:
        if node.tag == "VoteType":
            df, vts = doVoteType(node)
            for vt in vts:
                choice = ChoiceTup(*vt, ChoiceText)
                choice_tups.append(choice)
            if res is None:
                res = df
            else:
                res = res.join(df, how="left")             
    res['Total'] = res.sum(axis=1)     
    res.columns = pd.MultiIndex.from_product([[ChoiceText], res.columns])   
    return res, choice_tups
def doContest(root_node, pref='', verbosity=0):
    if verbosity > 0:
        print(node_msg(root_node, pref=pref+'  ', attribs='text'))       
    res = None
    ContestText = root_node.attrib['text']     
    contest_tups = []
    for node in root_node:
        if node.tag == "Choice":
            choice, tups = doChoice(node)              
            for tup in tups:
                contest_tup = ContestTup(*tup, ContestText)
                contest_tups.append(contest_tup)
            if res is None:
                res = choice
            else:
                res = res.join(choice, how='left')
    return (res, contest_tups)

In [5]:
xtree = et.parse(fpath)
senate_races = []
xroot = xtree.getroot()
tups = []
contests = {}
for i, node in enumerate(tqdm(xroot)): 
    if node.tag == 'Contest':
        if 'text' in node.attrib.keys():
            if 'US Senate'  in node.attrib['text'] or \
                'President' in node.attrib['text'] :
                contest = node.attrib['text']
                res, contest_tups = doContest(node, pref='')
                tups.extend(contest_tups)
                print(contest)
                contests[contest] = res
contests.keys()
contestsdf = pd.DataFrame(tups)
contestsdf['CountyName'] = contestsdf['CountyName'].apply(lambda x: x.upper())
contestsdf.groupby(by="Contest").sum()

  0%|                                                                                          | 0/303 [00:00<?, ?it/s]

President of the United States
US Senate (Perdue)


100%|██████████████████████████████████████████████████████████████████████████████| 303/303 [00:00<00:00, 1027.60it/s]

US Senate (Loeffler) - Special


,Votes
Contest,
President of the United States,4998482
US Senate (Loeffler) - Special,4914361
US Senate (Perdue),4952175


In [6]:
print(contestsdf['VoteType'].unique())
contestsdf.head(2)

['Election Day Votes' 'Absentee by Mail Votes' 'Advanced Voting Votes'
 'Provisional Votes']


,CountyName,Votes,VoteType,Choice,Contest
0,APPLING,1753,Election Day Votes,Donald J. Trump (I) (Rep),President of the United States
1,ATKINSON,716,Election Day Votes,Donald J. Trump (I) (Rep),President of the United States


In [7]:
grp_contests = contestsdf.groupby(by=['VoteType','Choice', 'Contest']).sum()
grp_contests.sort_values(by=['Contest', 'VoteType','Choice'],inplace=True)
grp_contests.reset_index(inplace=True)
grp_contests.head()

,VoteType,Choice,Contest,Votes
0,Absentee by Mail Votes,Donald J. Trump (I) (Rep),President of the United States,451157
1,Absentee by Mail Votes,Jo Jorgensen (Lib),President of the United States,16057
2,Absentee by Mail Votes,Joseph R. Biden (Dem),President of the United States,849729
3,Advanced Voting Votes,Donald J. Trump (I) (Rep),President of the United States,1419161
4,Advanced Voting Votes,Jo Jorgensen (Lib),President of the United States,25209


In [8]:
grp_contests.loc[grp_contests['Contest'].str.contains('President'),]
grp_contests.sort_values(by="Votes", inplace=True)
grp_contests.loc[grp_contests['Contest'].str.contains('Loeffler'),].tail(10)

,VoteType,Choice,Contest,Votes
37,Advanced Voting Votes,Deborah Jackson (Dem),US Senate (Loeffler) - Special,173820
19,Absentee by Mail Votes,Doug Collins (Rep),US Senate (Loeffler) - Special,185300
68,Election Day Votes,Raphael Warnock (Dem),US Senate (Loeffler) - Special,205110
59,Election Day Votes,Doug Collins (Rep),US Senate (Loeffler) - Special,211902
25,Absentee by Mail Votes,Kelly Loeffler (I) (Rep),US Senate (Loeffler) - Special,249090
65,Election Day Votes,Kelly Loeffler (I) (Rep),US Senate (Loeffler) - Special,298209
39,Advanced Voting Votes,Doug Collins (Rep),US Senate (Loeffler) - Special,581871
28,Absentee by Mail Votes,Raphael Warnock (Dem),US Senate (Loeffler) - Special,601522
45,Advanced Voting Votes,Kelly Loeffler (I) (Rep),US Senate (Loeffler) - Special,724577
48,Advanced Voting Votes,Raphael Warnock (Dem),US Senate (Loeffler) - Special,807005


In [9]:
presdf = contestsdf[contestsdf['Contest'].str.contains("President")]
presByCounty = presdf[["CountyName", "Votes"]].groupby("CountyName").sum()
presByCounty.sort_values(by="Votes", ascending=False, inplace=True)
presByCounty.head(4)

,Votes
CountyName,
FULTON,524659
GWINNETT,413865
COBB,393746
DEKALB,370804


In [10]:
loefflerdf = contestsdf[contestsdf['Contest'].str.contains("Loeffler")]
purduedf = contestsdf[contestsdf['Contest'].str.contains("Loeffler")]
if True:
    gdf = presdf[['VoteType', 'Votes', 'Contest']].groupby('VoteType').sum() 
    gdf["Pct"] = 100*np.round(gdf / gdf.sum(),3)
    print("Votes counted by Vote Type")
    print(gdf)
    print("------")
    print("Total votes counted (millions): {0}".format(gdf["Votes"].sum()/10.0**6))

Votes counted by Vote Type
                          Votes   Pct
VoteType                             
Absentee by Mail Votes  1316943  26.3
Advanced Voting Votes   2694879  53.9
Election Day Votes       975540  19.5
Provisional Votes         11120   0.2
------
Total votes counted (millions): 4.998482


In [11]:
votesByCounty = contestsdf[["CountyName", "Votes"]].groupby("CountyName").sum()
votesByCounty.sort_values(by="Votes", ascending=False, inplace=True)
votesByCounty.head(4)

,Votes
CountyName,
FULTON,1563044
GWINNETT,1227786
COBB,1173481
DEKALB,1105580


<span style="color:blue">
    
---
    
# Registered voters by County, Age, Race
    
https://sos.ga.gov/index.php/Elections/voter_registration_statistics
    
---
    
</span>

In [12]:
if True:
    dpath = "./data/"
    voterfile = "Active_Voters_by_Race_Gender_and_Age_as_of_November_1_2020.xlsx"
    voterpath = os.path.join(dpath, voterfile)
    if not os.path.isdir(dpath):
        warnings.warn("{0} is not a directory".format(dpath))
    if not os.path.isfile(voterpath):
        warnings.warn("No file {0} in dir {1}".format(voterfile, dpath))
    with open(voterpath) as vp:
        racedf = pd.read_excel(voterpath, nrows=7, engine='openpyxl').iloc[:,:2]
        racedf.columns = ["RaceCode", "RaceDescription"]
        racedf["RaceCode"] = racedf["RaceCode"].apply(lambda x: x[:-2])
        racedf.set_index("RaceCode", inplace=True)
        race = racedf.squeeze()
        race_dict = race.to_dict()
        print(race_dict)

{'AI': 'AMERICAN INDIAN OR ALASKAN NATIVE', 'AP': 'ASIAN OR PACIFIC ISLANDER', 'BH': 'BLACK NOT OF HISPANIC ORIGIN', 'HP': 'HISPANIC', 'WH': 'WHITE NOT OF HISPANIC ORIGIN', 'OT': 'OTHER', 'UK': 'UNKNOWN'}


In [13]:

votersdf = pd.read_excel(voterpath, skiprows=8, engine='openpyxl')
if votersdf.columns[-1] != "TOTAL VOTERS":
    votersdf = votersdf.iloc[:,:-1]
colnames = [re.sub("\s+", "-", c.strip()) for c in votersdf.columns]
votersdf.columns = colnames

votersdf.drop('COUNTY-ID', axis=1, inplace=True)    
votersdf = votersdf.loc[votersdf["COUNTY-NAME"].notna()]
colname_parts = [c.split("-") for c in votersdf.columns]
newnames = []

for parts in colname_parts:
    newparts = parts
    if parts[0]  in race_dict.keys():
        newparts = [race_dict[parts[0]].split(" ")[0]] + parts[1:]
    newnames.append("-".join(newparts))
votersdf.columns = newnames
votersdf.rename(columns={"COUNTY-NAME":"CountyName", 
                        'TOTAL-VOTERS':'TotalVoters'}, inplace=True)
votersdf['CountyName'] = votersdf['CountyName'].apply(lambda x: x.upper())
print("{0} rows {1} cols".format(*votersdf.shape))
votersdf.head(2)

1594 rows 24 cols


,CountyName,VOTER-AGE-RANGE,AMERICAN-MALE-VOTERS,AMERICAN-FEMALE-VOTERS,AMERICAN-UNKNOWN-VOTERS,ASIAN-MALE-VOTERS,ASIAN-FEMALE-VOTERS,ASIAN-UNKNOWN-VOTERS,BLACK-MALE-VOTERS,BLACK-FEMALE-VOTERS,...,WHITE-MALE-VOTERS,WHITE-FEMALE-VOTERS,WHITE-UNKNOWN-VOTERS,OTHER-MALE-VOTERS,OTHER-FEMALE-VOTERS,OTHER-UNKNOWN-VOTERS,UNKNOWN-MALE-VOTERS,UNKNOWN-FEMALE-VOTERS,UNKNOWN-UNKNOWN-VOTERS,TotalVoters
0,APPLING,18-24,2,1,0,3,5,0,120,110,...,436,429,1,5,19,0,95,60,2,1374
1,APPLING,25-29,0,0,0,4,1,0,81,89,...,282,298,1,3,7,0,38,30,0,866


In [14]:
gcols = ["CountyName"]
votersByCounty = votersdf[["CountyName", 'TotalVoters' ]].groupby(by=gcols).sum()
votersByCounty.sort_values(by="TotalVoters", ascending=False, inplace=True)
votersByCounty.dtypes

TotalVoters    int64
dtype: object

<span style="color:blue">
    
---
    
# Combining Voters and Votes by County
    
---
    
</span>

In [15]:
print(presByCounty.index.values[:10])
print(votersByCounty.index.values[:10])


['FULTON' 'GWINNETT' 'COBB' 'DEKALB' 'CHEROKEE' 'CHATHAM' 'FORSYTH'
 'HENRY' 'CLAYTON' 'HALL']
['FULTON' 'GWINNETT' 'DEKALB' 'COBB' 'CHATHAM' 'CLAYTON' 'CHEROKEE'
 'HENRY' 'FORSYTH' 'RICHMOND']


In [16]:
presCombo = presByCounty.join(votersByCounty)
presCombo["PctVoted"] = np.round(100*presCombo['Votes'] / presCombo['TotalVoters'],3)
presCombo.sort_values(by="Votes", inplace=True)
presCombo.tail(10)

,Votes,TotalVoters,PctVoted
CountyName,,,
HALL,90523,128535,70.427
CLAYTON,112344,194338,57.809
HENRY,122742,172241,71.262
FORSYTH,129305,164279,78.711
CHATHAM,133420,202999,65.724
CHEROKEE,144830,190605,75.984
DEKALB,370804,547802,67.689
COBB,393746,537659,73.233
GWINNETT,413865,582917,70.999


<span style="color:blue">
    
---
    
# Voter Registration
    
### File has a record for *each* registered voter
    
https://elections.sos.ga.gov/Elections/voterhistory.do

---
    
</span>

In [17]:
def get_individual_voter_reg_files(download_dir, verbosity=0):
    if verbosity > 0:
        print("{0} download_dir= {1}, <{2}>".format(whoami(), download_dir,
                                                   datetime.datetime.now()))
    options = chrome_options_download_dir(download_dir)    
    chromedriver_path = os.path.join('chromedriver.exe')
    driver = webdriver.Chrome(executable_path=chromedriver_path, options=options)
    url = "https://elections.sos.ga.gov/Elections/voterhistory.do"
    driver.get(url)
    # find year selector and select 2020
    year_select = Select(driver.find_element_by_id("nbElecYear"))
    year_select.select_by_value("2020")
    # find the election selector and select 11/03/2020
    elec_select = Select(driver.find_element_by_id("idElection"))
    elec_select.select_by_visible_text("11/03/2020 - NOVEMBER 3, 2020 GENERAL/SPECIAL ELECTION")
    # find the elecCat selector and select StaeWide
    cat_select = Select(driver.find_element_by_id("cdElecCat"))
    cat_select.select_by_visible_text("STATE WIDE")
    time.sleep(0.5)
    # click on button to download 2020 file
    if verbosity > 0:
        print("  downloading 2020 file")
    driver.find_element_by_partial_link_text("2020").click()
    time.sleep(0.5)
    fname = "2020.zip"
    max_wait_secs = 9
    file_there = False
    start_dt = datetime.datetime.now()
    if verbosity > 0:
        print("looking for file {0} in {1}".format(fname, download_dir))
    while not file_there:
        files = os.listdir(download_dir)
        if fname in files:
            if verbosity > 0:
                print("  Found it '{0}'".format(fname))
            file_there = True
            break
        cur_dt = datetime.datetime.now()
        if verbosity > 0:
            print("  Not there yet <{0}>".format(cur_dt))
        time.sleep(1)
        if (cur_dt - start_dt).total_seconds() > max_wait_secs:
            if verbosity > 0:
                print("{0} not in {1}, can't wait any longer".format(fname, download_dir))
            break
    if file_there:
        if verbosity > 0:
            print(" Unzipping {0} to {1}".format(fname, download_dir))
        fpath = os.path.join(download_dir, fname)
        zpath = os.path.join(download_dir, fname.split(".")[0])
        with zipfile.ZipFile(fpath, 'r') as zip_ref:
            zip_ref.extractall(zpath)    
    #click on button to download County files
    if verbosity > 0:
        print("  downloading county file")    
    driver.find_element_by_partial_link_text("County Number").click() 
    time.sleep(1)
    if verbosity > 0:
        print("Done {0}".format(datetime.datetime.now()))
    
    

In [18]:
data_path = "./data"
elems = os.listdir(data_path)
if "2020" not in elems:
    get_individual_voter_reg_files(download_dir=data_path, verbosity=1)
data_path = "./data/2020"    
if not os.path.isdir(data_path):
    warnings.warn("2020 data not there, Somthing wrong")
else:
    elems_2020 = os.listdir(data_path)
    fname = "2020.TXT"
    fpath = os.path.join(data_path, fname)
    if fname not in elems_2020:
        warnings.warn("{0} not found".format(fname))
    elif not os.path.isfile(fpath):
        warnings.warn("{0} not a file".format(fname))
    else:
        fpath = os.path.join(data_path, fname)
        print("Detailed registerd voter data for 2020 in the file '{0}' in the directory {1}".format(fname, dpath))

Detailed registerd voter data for 2020 in the file '2020.TXT' in the directory ./data/


In [19]:
ElectionType = {
'001':'GENERAL PRIMARY',
'002':'GENERAL PRIMARY RUN-OFF',
'003':'GENERAL',
'004':'GENERAL ELECTION RUNOFF',
'005':'SPECIAL ELECTION',
'006':'SPECIAL RUNOFF',
'007':'NON- PARTISAN',
'008':'SPECIAL/NON-PARTISAN',
'009':'RECALL',
'010':'PPP'
}
#SPECIAL PRIMARY RUNOFF
#SPECIAL PRIMARY

## Pandas has a method for reading in fixed width data

panda.read_fwf


In [20]:
# Pandas has a method for readin in fixed width data
# the pandas.read_fwf
# tried it, not working so well, need to tell it no header
#for now still use the brute force method below
if False:
    field_lens = {'county_num':3,
            'registration_num':8,
            'election_date':8,
            'election_type': 3,
            'party': 2,
            'absentee':1,
            'provisional':1,
            'supplemental':1}
    x = pd.read_fwf(fpath, widths = field_lens.values(), names=field_lens.keys())
    x.head()

In [21]:
with open(fpath, mode='r') as fp:
    txt = fp.read()
voter_lines = txt.split('\n')
minlen = min([len(l) for l in voter_lines])
maxlen = max([len(l) for l in voter_lines])
print("{0} had {1} lines, minlen {2}, maxlen {3}".format(fname, len(voter_lines), minlen, maxlen))

2020.TXT had 8454089 lines, minlen 0, maxlen 27


In [ ]:
import numpy as np

DELIMITER = (20, 10, 10, 20, 10, 10, 20)

idx = np.cumsum([0] + list(DELIMITER))
slices = [slice(i, j) for (i, j) in zip(idx[:-1], idx[1:])]

def parse(line):
    return [line[s] for s in slices]

In [27]:
def parse_voter_lines(lines=None):
    from collections import namedtuple
    from collections import OrderedDict
    field_dict = {'county_num':3,
                'registration_num':8,
                'election_date':8,
                'election_type': 3,
                'party': 2,
                'absentee':1,
                'provisional':1,
                'supplemental':1}
    field_widths = field_dict.values()
    field_names =  field_dict.keys()

    #print(field_lens.keys())
    VoterRec = namedtuple("VoterRec", field_dict.keys() )
    expected_len = sum(field_dict.values())

    # parse slices
    stups = []
    def parse_slices(line, slices):
        return [line[s] for s in slices]      
    widths = field_dict.values()
    idx = np.cumsum([0] + list(widths))
    slices = [slice(i, j) for (i, j) in zip(idx[:-1], idx[1:])]    
    for i, line in enumerate(tqdm(lines)):
        if len(line) != expected_len:
            msg = "{0} len={1}, line={2}".format(i, len(line), line)
            continue
        data = parse_slices(line, slices)
        vr = VoterRec(*data)
        stups .append(vr)        
        
    # parse fields    
    ftups = []    
    def parse_fields(line, field_dict):
        start = 0
        data = {}
        for k in field_dict.keys():
            width = field_dict[k]
            elem = line[start:(start+width)]
            data[k] = elem
            start += width
        return data
    for i, line in enumerate(tqdm(lines)):
        if len(line) != expected_len:
            msg = "{0} len={1}, line={2}".format(i, len(line), line)
            continue
        tup = parse_fields(line, field_dict)
        vr = VoterRec(**tup)
        ftups .append(vr)

    fdf = pd.DataFrame(ftups)
    sdf = pd.DataFrame(stups)
    return fdf, sdf, ftups, stups

regdf1, regdf2, ftups, stups = parse_voter_lines(lines=voter_lines)
print(tups[:2])


100%|████████████████████████████████████████████████████████████████████| 8454089/8454089 [00:30<00:00, 276145.56it/s]


[ContestTup(CountyName='Appling', Votes=1753, VoteType='Election Day Votes', Choice='Donald J. Trump (I) (Rep)', Contest='President of the United States'), ContestTup(CountyName='Atkinson', Votes=716, VoteType='Election Day Votes', Choice='Donald J. Trump (I) (Rep)', Contest='President of the United States')]


In [28]:
for df in (regdf1, regdf2):
    print("{0}  {1}".format(df.shape[0], df.shape[1]))
    print(df.head(2))

8454088  8
  county_num registration_num election_date election_type party absentee  \
0        067         02788924      20200324           005              N   
1        067         05761105      20200324           005              N   

  provisional supplemental  
0           N            N  
1           N            N  
8454088  8
  county_num registration_num election_date election_type party absentee  \
0        067         02788924      20200324           005              N   
1        067         05761105      20200324           005              N   

  provisional supplemental  
0           N            N  
1           N            N  


In [32]:
etype = '003'
gendf = regdf1.loc[regdf1['election_type'] == etype]
print("{0} rows for election type = {1}".format(gendf.shape[0], etype))
print(gendf.head(2))
cnames = ['absentee', 'provisional', 'supplemental']

for name in tqdm(cnames, smoothing=0.2):
    print("converting '{0}'".format(name))
    temp = gendf[name].apply(lambda x: int(x=='Y'))
    print("{0} {1}".format(len(temp), gendf.shape[0]))
    gendf[name] = temp
gendf['sum'] = gendf[cnames].sum(axis=1) 
gendf['other'] = gendf['sum'].apply(lambda x: int(x==0))
print(gendf.head(2))

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

5007405 rows for election type = 003
     county_num registration_num election_date election_type party absentee  \
5419        019         00591608      20201103           003              Y   
5420        027         00430264      20201103           003              Y   

     provisional supplemental  
5419           N            N  
5420           N            N  
converting 'absentee'
5007405 5007405


<ipython-input-32-635951d2033b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 33%|████████████████████████████                                                        | 1/3 [00:01<00:03,  1.98s/it]

converting 'provisional'
5007405 5007405


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:03<00:01,  1.98s/it]

converting 'supplemental'
5007405 5007405


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.97s/it]
<ipython-input-32-635951d2033b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


     county_num registration_num election_date election_type party  absentee  \
5419        019         00591608      20201103           003               1   
5420        027         00430264      20201103           003               1   

      provisional  supplemental  sum  other  
5419            0             0    1      0  
5420            0             0    1      0  


<ipython-input-32-635951d2033b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [35]:
vtypes = cnames + ['other']
gendf[vtypes].value_counts()

absentee  provisional  supplemental  other
1         0            0             0        4018963
0         0            0             1         981659
          1            0             0           6477
          0            1             0            165
1         1            0             0            124
0         1            1             0             12
1         1            1             0              4
          0            1             0              1
dtype: int64

In [46]:
gendf['other_int'] = [1 if gendf[]]
gendf[inames].sum()
gendf.head(2)

<ipython-input-46-394ad742e9d4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,county_num,registration_num,election_date,election_type,party,absentee,provisional,supplemental,absentee_int,provisional_int,supplemental_int
5419,019,00591608,20201103,003,,Y,N,N,1,0,0
5420,027,00430264,20201103,003,,Y,N,N,1,0,0


In [49]:
print("millions of rows  {0}".format(gendf.shape[0]/10.0**6))
gendf['typesum'] = gendf[inames].sum(axis=1)
gendf.head()
print(gendf.shape[0] - gendf.loc[gendf['typesum'] != 1].shape[0])
print(gendf.shape)
#print(gendf[inames].sum().sum())

millions of rows  5.007405


<ipython-input-49-65212c856066>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


4025605
(5007405, 12)


In [55]:
allcols = ['county_num', 'election_date', 'party', 'absentee_int', 'provisional_int', 'supplemental_int']
gcols = ['county_num', 'election_date', 'party']           
gdf = gendf[allcols].groupby(by=gcols).sum()


In [56]:
gdf

absentee_int  provisional_int  \
county_num election_date party                                  
001        20201103                     6300                2   
           20201204                        1                0   
002        20201103                     2184                0   
003        20201103                     4093                0   
004        20201103                     1114                2   
...                                      ...              ...   
155        20201103                    24838               64   
156        20201103                     2250                0   
157        20201103                     3790                5   
158        20201103                     2937                0   
159        20201103                     5926                0   

                                supplemental_int  
county_num election_date party                    
001        20201103                            0  
           20201204                            0  
002        20201103                            0  
003        20201103                            0  
004        20201103                            0  
...                                          ...  
155        20201103                            0  
156        20201103                            0  
157        20201103                            0  
158        20201103                           34  
159        20201103                            0  

[171 rows x 3 columns]

In [ ]:
fpath = os.path.join('.',"data")
fname = "County_Number_List.pdf"
# Creating a pdf file object.
pdf = open(fpath, mode="rb")
 
# Creating pdf reader object.
pdf_reader = PyPDF2.PdfFileReader(pdf)
 
# Checking total number of pages in a pdf file.
print("Total number of Pages:", pdf_reader.numPages)
 
# Creating a page object.
page = pdf_reader.getPage(200)
 
# Extract data from a specific page number.
print(page.extractText())
 
# Closing the object.
pdf.close()

In [ ]:
# Election Types
election_types = {
    '001':'GENERAL PRIMARY',
    '002':'GENERAL PRIMARY RUN-OFF',
    '003':'GENERAL',
    '004':'GENERAL ELECTION RUNOFF',
    '005':'SPECIAL ELECTION',
    '006':'SPECIAL RUNOFF',
    '007':'NON- PARTISAN',
    '008':'SPECIAL/NON-PARTISAN',
    '009': ' RECALL',
    '010': 'PPP'
    #SPECIAL PRIMARY RUNOFF
    #SPECIAL PRIMARY
    }

In [ ]:
gcols = ['election_date', 'election_type']
df.groupby(gcols).count()

In [ ]:
fpath = os.path.join(os.getcwd(),"data")
fname = "County_Number_List.pdf"
if not os.path.isfile(os.path.join(fpath, fname)):
    if not os.path.isdir(fpath):
        warnings.warn("{0} not a directory".format(fpath))
    elif fname not in os.listdir(fpath):
        warnings.warn("{0} not a file in {1}".format(fname, fpath))
table = tabula.read_pdf(fpath,pages=1)
table[0]